# Thông tin sinh viên

Họ tên: Nguyễn Nguyên Khôi

MSSV: 21521009

Bài tập số 4

Ngày thực hiện 5-4-2023

# Yêu cầu 1: Phân lớp ảnh Pedestrian vs NonPedestrian

In [1]:
import numpy as np
from numpy.linalg import norm
import pandas as pd
import cv2 as cv
import os
import matplotlib.pyplot as plt
from scipy.spatial import distance
from scipy import ndimage
from skimage import feature
from google.colab import drive
from google.colab.patches import cv2_imshow
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [2]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [3]:
dir_dataset = "gdrive/MyDrive/CS231.N22.KHCL/Baitap/Baitap4/Dataset/Pedestrian"
dir_pedestrian = os.path.join(dir_dataset, "Pedestrian")
dir_nonpedestrian = os.path.join(dir_dataset, "NonPedestrian")

In [4]:
def compute_hog_train(pathfilename):
  img = cv.imread(pathfilename,0)
  img = cv.resize(img, (128, 256))
  (hog, hog_image) = feature.hog(img, orientations=9, pixels_per_cell=(8, 8), 
                                 cells_per_block=(2, 2), block_norm='L2-Hys', 
                                 visualize=True, transform_sqrt=True)

  return hog

In [5]:
def CalHist(pathfilename):
  img = cv.imread(pathfilename,0)
  hist = cv.calcHist([img], [0], None, [256], [0,256])
  size = img.shape[0]*img.shape[1]
  hist = hist / size
  return hist

In [6]:
def EF(pathfilename, op):
  if op == 0:
    return CalHist(pathfilename)
  else: 
    return compute_hog_train(pathfilename)

In [7]:
def ReadData(path_to_files, op, l):
  features = []
  labels = []
  for file_name in os.listdir(path_to_files):
    # Take label of the image
    # -> Insert into list of labels
    label = l
    if label == 'pedestrian':
      labels.append(0)
    else:
      labels.append(1)
    # Calculate image feature
    feature = EF(path_to_files + '/'+ file_name, op)
    print(feature)
    # -> Insert into list of features
    features.append(feature)

  features = np.array(features)
  labels = np.array(labels)
  return features, labels

In [8]:
X_ped, y_ped = ReadData(dir_pedestrian, 0, "pedestrian")
X_nonped, y_nonped = ReadData(dir_nonpedestrian, 0, "nonpedestrian")
X = np.concatenate((X_ped, X_nonped), axis=0)
y = np.concatenate((y_ped, y_nonped), axis=0)

Streaming output truncated to the last 5000 lines.
 [0.0024625]
 [0.0027375]
 [0.0024875]
 [0.0028875]
 [0.0022625]
 [0.00255  ]
 [0.002525 ]
 [0.00255  ]
 [0.00235  ]
 [0.0024625]
 [0.0025375]
 [0.002825 ]
 [0.00255  ]
 [0.002525 ]
 [0.0030125]
 [0.00295  ]
 [0.00315  ]
 [0.003175 ]
 [0.0028375]
 [0.002775 ]
 [0.00285  ]
 [0.0028875]
 [0.0030875]
 [0.0028875]
 [0.0027125]
 [0.0026   ]
 [0.002625 ]
 [0.0023875]
 [0.002875 ]
 [0.0026375]
 [0.0024625]
 [0.002475 ]
 [0.002525 ]
 [0.0025625]
 [0.0026875]
 [0.0025   ]
 [0.002725 ]
 [0.0026875]
 [0.00285  ]
 [0.0025   ]
 [0.00245  ]
 [0.0028125]
 [0.0025875]
 [0.0028375]
 [0.0027625]
 [0.0029   ]
 [0.002825 ]
 [0.0030875]
 [0.0031375]
 [0.0033125]
 [0.0031   ]
 [0.003325 ]
 [0.003775 ]
 [0.003925 ]
 [0.0036875]
 [0.004325 ]
 [0.0043   ]
 [0.00445  ]
 [0.0047   ]
 [0.005075 ]
 [0.0050625]
 [0.0057   ]
 [0.0061875]
 [0.0062125]
 [0.00665  ]
 [0.006675 ]
 [0.007225 ]
 [0.00755  ]
 [0.0074875]
 [0.0077625]
 [0.0079375]
 [0.0074   ]
 [0.0073375]


In [9]:
print(X_ped.shape)

(20, 256, 1)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

In [ ]:
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)

Shape of X_train: (32, 256)
Shape of X_test: (8, 256)


In [ ]:
accuracy = []
k_list = [i for i in range(1, 33)]
for k in k_list:
  knn_model = KNeighborsClassifier(n_neighbors=k)
  knn_model.fit(X_train, y_train)
  acc = knn_model.score(X_test, y_test)
  accuracy.append(acc)

In [ ]:
result = pd.DataFrame(list(zip(k_list, accuracy)), columns =["K", "Accuracy on test set"])
result

,K,Accuracy on test set
0,1,0.625
1,2,0.500
2,3,0.625
3,4,0.750
4,5,0.750
5,6,0.750
6,7,0.750
7,8,0.750
8,9,0.625
9,10,0.500


In [ ]:
print(f"The best value of k is k = {accuracy.index(max(accuracy))+1}, accuracy = {max(accuracy)}")

The best value of k is k = 4, accuracy = 0.75


# Yêu cầu 2: Áp dụng Linear Classification cho đặc trưng Histogram

In [10]:
X_ped, y_ped = ReadData(dir_pedestrian, 1, "pedestrian")
X_nonped, y_nonped = ReadData(dir_nonpedestrian, 1, "nonpedestrian")
X = np.concatenate((X_ped, X_nonped), axis=0)
y = np.concatenate((y_ped, y_nonped), axis=0)

[0.24282298 0.10761554 0.13477374 ... 0.03886791 0.00566404 0.03793991]
[0.01155229 0.01893771 0.01360101 ... 0.04083888 0.0320725  0.0494684 ]
[0.23512484 0.04734355 0.20549995 ... 0.00326347 0.         0.        ]
[0.07176033 0.01317238 0.03222053 ... 0.00408048 0.00296268 0.00069205]
[0.21139611 0.2397594  0.2166577  ... 0.03494488 0.04146066 0.01710025]
[0.30753978 0.01074863 0.00670653 ... 0.06885738 0.05509989 0.05091911]
[0.05313982 0.00502759 0.01338696 ... 0.16722617 0.20724584 0.0335596 ]
[0.1615567  0.         0.1370854  ... 0.13104733 0.0346572  0.02483593]
[0.01726975 0.05114221 0.11886354 ... 0.0671246  0.13667288 0.12630309]
[0.35177051 0.11564856 0.13332313 ... 0.         0.00900196 0.        ]
[0.64492681 0.         0.0124133  ... 0.01784927 0.         0.        ]
[0.24674545 0.07220096 0.11960421 ... 0.00795513 0.0042865  0.        ]
[0.26470775 0.10820376 0.14674247 ... 0.01378359 0.00714735 0.01020006]
[0.26904398 0.01157379 0.10616858 ... 0.01387948 0.00282287 0.01

In [11]:
X_ped.shape

(20, 16740)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1]))

In [ ]:
log_reg_model = LogisticRegression(random_state=42)
log_reg_model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [ ]:
# accuracy on test set
acc = log_reg_model.score(X_test, y_test)
print(acc)

0.875
